In [1]:
import import_ipynb
import torch
import gpytorch 
import math
import copy 
from buildModel import ExactGPmodel


In [2]:
def optimization(llh,model,Xtrain,ytrain,tolerance_grad,training_iter = 200, learning_rate=0.01,numPrint = 100,ifPrint = True):
    optimizer = torch.optim.LBFGS(model.parameters(),lr = learning_rate, history_size=15,line_search_fn='strong_wolfe',tolerance_grad=tolerance_grad)
    
    #marginal likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(llh,model)

    for i in range(training_iter):
        def closure():
            optimizer.zero_grad()
            output = model(Xtrain)
            loss = -mll(output, ytrain)
            loss.backward()
            return loss
        optimizer.step(closure)
        loss = optimizer.step(closure)
        if ifPrint is True:
            if (i+1) % numPrint == 0:
                log_message = 'Iter %d/%d loss: %.3f' % (i + 1, training_iter, loss.item())
                lengthscale_values = model.covar_module.base_kernel.lengthscale[0]   
                for idx, lengthscale in enumerate(lengthscale_values):
                    log_message += ' W%d: %.3f' % (idx + 1, math.log(1 / lengthscale.item()))
                noise_value = model.likelihood.noise.item()
                log_message += ' noise: %.4f' % noise_value
                print(log_message)
    w = torch.log(1/model.covar_module.base_kernel.lengthscale[0])
    return w

In [3]:

def Inference(Xtrain,Ytrain,training_iter=200,learning_rate= 0.01,ifPrint = True,numPrint = 20):

    N = Ytrain.size()[1] # number of dimension
    [numData,numFeatures] = Xtrain.size()
    W = torch.zeros(N,numFeatures) # create the weighted adjacency matrix W
    ini = 10*torch.ones([1,numFeatures])
    
    for n in range(N):
        print("processing for nodes" + str(n+1))
        tolerance_grad = 1e-6
        # target variable for each node
        ytrain = Ytrain[:,n]
        # build model here 
    
        
        while True:
            try:
                # initialize likelihood and model
                llh = []
                model = []
                
                llh = gpytorch.likelihoods.GaussianLikelihood(noise_constraint=gpytorch.constraints.GreaterThan(1e-4))
                llh.noise_covar.noise = ytrain.std()
        
                model = ExactGPmodel(Xtrain, ytrain, llh)       
                model.covar_module.base_kernel.lengthscale = ini
                model.covar_module.outputscale = ytrain.std()
        
                # in training mode
                model.train()
                llh.train()
                
                # optimization
                w = optimization(llh,model,Xtrain,ytrain,tolerance_grad)
                break
            except Exception as e:
                if tolerance_grad<1:
                    tolerance_grad = 10*tolerance_grad
                    print("************************************************************")
                    print("tolerance of gradient is "+ str(tolerance_grad))
                    print("#######################################################")
                else:
                    print("Failed for optimization after increasing the tolerance of gradient")
                    break                    
        W[n,:] = w
    return W